# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('5&dummies.csv')

In [3]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       40000 non-null  int64  
 1   city_rank        40000 non-null  float64
 2   rating           40000 non-null  float64
 3   price            40000 non-null  float64
 4   num_of_reviews   40000 non-null  float64
 5   num_of_cuisines  40000 non-null  int64  
 6   Amsterdam        40000 non-null  int64  
 7   Athens           40000 non-null  int64  
 8   Barcelona        40000 non-null  int64  
 9   Berlin           40000 non-null  int64  
 10  Bratislava       40000 non-null  int64  
 11  Brussels         40000 non-null  int64  
 12  Budapest         40000 non-null  int64  
 13  Copenhagen       40000 non-null  int64  
 14  Dublin           40000 non-null  int64  
 15  Edinburgh        40000 non-null  int64  
 16  Geneva           40000 non-null  int64  
 17  Hamburg     

In [27]:
df = df.select_dtypes(exclude='object')
df.columns = ['ranking', 'rating','num_of_reviews']
#df = df[df['num_of_reviews'] <= 50]
#df.num_of_reviews.fillna(12, inplace=True)
#df['num_of_reviews'] = df.num_of_reviews.fillna(1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ranking         40000 non-null  float64
 1   rating          40000 non-null  float64
 2   num_of_reviews  37457 non-null  float64
dtypes: float64(3)
memory usage: 937.6 KB


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [33]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
train_df = df.dropna(axis=0)[df.num_of_reviews < 150]
print(train_df.info())
X = train_df.drop(['num_of_reviews'], axis = 1)
y = train_df['num_of_reviews']
Z = df[df.num_of_reviews.isna()].drop(['num_of_reviews'], axis = 1)
Z.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29815 entries, 1 to 39998
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ranking         29815 non-null  float64
 1   rating          29815 non-null  float64
 2   num_of_reviews  29815 non-null  float64
dtypes: float64(3)
memory usage: 931.7 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2543 entries, 8 to 39989
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ranking  2543 non-null   float64
 1   rating   2543 non-null   float64
dtypes: float64(2)
memory usage: 59.6 KB


<ipython-input-33-976406c29b93>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_df = df.dropna(axis=0)[df.num_of_reviews < 150]


In [34]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [35]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [36]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [38]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
z_pred = round(regr.predict(Z))
df.fillna(z_pred, inplace=True)
df.info

array([ 40.88178571,  21.07      ,  15.21      , ..., 100.41      ,
        10.94583333,   2.56      ])

In [41]:
df.fillna(z_pred, inplace=True)
df.info()

ValueError: invalid fill value with a <class 'numpy.ndarray'>

In [39]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 29.16341110519478
